In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from lib import load_data
%matplotlib inline

In [117]:
users_df, costs_df, relations_df, visits_df, orders_df = load_data()

C:\Users\myz\AppData\Local\Continuum\anaconda3\envs\idp3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


## Задание №1:	
## С помощью инструментов R или Python посчитать следующие метрики:

### ROI для 0-го, 6-го 14-го и 30-го дней жизни пользователя на сайте в разрезе маркетинговых каналов (chanel из relations).

#### Join Users with Realtions to get chanels per user and Costs to get cost per user

In [119]:
users_df = users_df.merge(relations_df, how='left', on=['name', 'id_partner'])
users_df = users_df.merge(costs_df, how='inner', left_on=['name', 'id_partner', 'Reg_date'], 
                                        right_on=['campaign', 'id_partner', 'date_created'])
users_df.set_index('id', inplace=True)

In [120]:
users_df['cost_per_user'] = users_df.groupby(['name', 'id_partner', 'Reg_date'])['Costs'].transform(
    lambda x: np.array(x)[0]/len(x))
users_df.head()

,Reg_date,id_partner,name,chanel,Costs,campaign,date_created,cost_per_user
id,,,,,,,,
7043369,2017-10-25,Clickdealer,-1,Clickdealer,10.0,-1,2017-10-25,10.0
6700130,2017-10-02,Advidi_email,-1,Advidi_email,20.0,-1,2017-10-02,10.0
6700181,2017-10-02,Advidi_email,-1,Advidi_email,20.0,-1,2017-10-02,10.0
5656121,2017-08-04,adsimilis_signup,-1,Adsimilis,9.0,-1,2017-08-04,9.0
5829186,2017-08-14,adsimilis_signup,-1,Adsimilis,9.0,-1,2017-08-14,9.0


#### Join Orders with Users to get Registration date in Orders

In [121]:
orders_df = orders_df.merge(users_df['Reg_date'].to_frame(), how='inner', left_on='id_user', right_index=True)
orders_df.head()

,id_user,Order Date,Amount,Order ID,Reg_date
0,4618612,2017-08-17 02:08:40,2.6312,23732390,2017-06-01
1,4618612,2017-08-17 02:29:26,17.5912,23732304,2017-06-01
25,4619702,2017-06-06 23:00:58,8.7912,12557068,2017-06-01
89,4620243,2017-06-01 04:19:23,2.6312,12441497,2017-06-01
90,4620243,2017-06-01 05:55:59,61.5912,12442776,2017-06-01


In [125]:
def roi_on_day(day, users_df, orders_user_reg_df):
    users_ids = users_df[users_df['Reg_date'] < np.max(users_df['Reg_date']) + np.timedelta64(-day - 1, 'D')].index
    orders_for_roi_mask = orders_user_reg_df['Order Date'] < orders_user_reg_df['Reg_date'] + np.timedelta64(day + 1, 'D')
    orders_for_roi_mask = orders_for_roi_mask & orders_user_reg_df['id_user'].isin(users_ids)
    cost_order_amount_per_user_df = orders_user_reg_df[orders_for_roi_mask].groupby('id_user').agg(
        {'Amount': 'sum'})

    order_amount_day_label = 'order_amount_day_' + str(day)
    users_df[order_amount_day_label] = cost_order_amount_per_user_df['Amount']
    users_df[order_amount_day_label].fillna(0, inplace=True)
    
    cost_amout_per_channel_df = users_df.groupby('chanel').agg({'cost_per_user': 'sum', order_amount_day_label: 'sum'})
    roi = 100 * (cost_amout_per_channel_df[order_amount_day_label] 
        - cost_amout_per_channel_df['cost_per_user'])/cost_amout_per_channel_df['cost_per_user']
    return roi

## Answer

In [131]:
roi = pd.DataFrame({'roi_0': roi_on_day(0, users_df, orders_df)})
roi['roi_14'] =  roi_on_day(14, users_df, orders_df)
roi['roi_30'] =  roi_on_day(30, users_df, orders_df)
roi

,roi_0,roi_14,roi_30
chanel,,,
10,-98.519317,-85.270184,-75.770628
15,-100.000000,-95.921294,-77.950570
29,-98.260524,-82.608193,-63.968284
32,-100.000000,-100.000000,-100.000000
37,-99.692393,-66.302014,-36.474880
52,-100.000000,-100.000000,-100.000000
7,-97.809828,-88.614134,-83.541003
7click_email,-98.137624,-79.727928,-58.499442
8,-100.000000,-96.409596,-76.232664


## Стоимость за пользователя, который совершил свою первую оплату в первые 14, 30 и 60 дней жизни пользователя на сайте в разрезе маркетинговых партнеров (id_partner).

In [184]:
def user_cost_paid_on_day(day, users_df, first_orders):
    users_mask = np.isin(users_df.index.values, first_orders.index.values)
    users_mask &= users_df['Reg_date'] < np.max(users_df['Reg_date']) + np.timedelta64(-day - 1, 'D')
    user_cost_per_partner = users_df[users_mask].groupby('id_partner').agg({'cost_per_user': 'mean'})
    return user_cost_per_partner
#     orders_mask = orders_user_reg_df['Order Date'] < orders_user_reg_df['Reg_date'] + np.timedelta64(day + 1, 'D')
#     orders_mask = orders_mask & orders_user_reg_df['id_user'].isin(users_ids)
#     user_ids_that_ordered = orders_df[orders_mask]['id_user'].unique()
#     costs = users_df[users_df.index.isin(user_ids_that_ordered)].groupby('id_partner').agg({'cost_per_user': lambda x: x.sum()/len(x)})
#     return costs['cost_per_user'] 

In [185]:
first_orders = orders_df.groupby('id_user').agg({'Order Date': lambda x: np.array(x.sort_values())[0]})
print('Average cost paid for user per partners in first {} days of users lifetime:'.format(14))
print(user_cost_paid_on_day(14, users_df, first_orders))

Average cost paid for user per partners in first 14 days of users lifetime:
                           cost_per_user
id_partner                              
10                              6.453358
1020                            3.301604
1020_mobile                     1.317024
1020n                           3.041687
14186                          20.762711
15                              4.724168
29                              5.363911
37                              3.942065
7                               2.213143
7click_email                    4.915420
9                               9.533036
ADC                             1.687500
Ad4date                         6.371220
Ad4date_mobile                  2.772727
Adsimilis                       7.002381
Adtrafico                       4.630768
Adtrafico_mobile                3.034021
Advidi                          6.907041
Affiliaxe_mobile                2.346079
Affiliaxe_non_adult_valid       5.953166
Beeclick              

In [186]:
print('Average cost paid for user per partners in first {} days of users lifetime:'.format(30))
print(user_cost_paid_on_day(30, users_df, first_orders))

Average cost paid for user per partners in first 30 days of users lifetime:
                           cost_per_user
id_partner                              
10                              6.453358
1020                            3.301604
1020_mobile                     1.317024
1020n                           3.041687
14186                          19.589086
15                              4.724168
29                              5.363911
37                              3.942065
7                               2.213143
7click_email                    4.883521
9                               6.629070
ADC                             1.687500
Ad4date                         6.371220
Ad4date_mobile                  2.772727
Adsimilis                       7.002381
Adtrafico                       4.630768
Adtrafico_mobile                3.034021
Advidi                          6.907041
Affiliaxe_mobile                2.346079
Affiliaxe_non_adult_valid       5.953166
Beeclick              

In [187]:
print('Average cost paid for user per partners in first {} days of users lifetime:'.format(60))
print(user_cost_paid_on_day(60, users_df, first_orders))

Average cost paid for user per partners in first 60 days of users lifetime:
                           cost_per_user
id_partner                              
10                              6.198840
1020                            3.301604
1020_mobile                     1.317024
1020n                           3.374733
14186                          22.184431
7                               2.279589
7click_email                    4.789082
9                               9.743534
ADC                             1.687500
Ad4date                         6.371220
Ad4date_mobile                  2.772727
Adsimilis                       7.002381
Adtrafico                       4.630768
Adtrafico_mobile                3.034021
Advidi                          6.856096
Affiliaxe_mobile                2.288005
Affiliaxe_non_adult_valid       5.846434
Beeclick                        8.359081
Beeclick_mobile                 3.970502
CPATrend_mobile                 3.583877
CPATrend_non_adult_val

### % пользователей, которые вернулись на сайт хотя бы раз в течении первой недели своей жизни в разрезе маркетинговых каналов (chanel из relations).

In [201]:
def pct_users_return_on_day(day, users_df):
    visits_mask = users_df['Visit_date'] < users_df['Reg_date'] + np.timedelta64(day + 1, 'D')
    visits_mask = visits_mask & ~users_df['Visit_date'].isnull()
    visited_mask_label = 'mask_visited_till_day_'+str(day)
    users_df[visited_mask_label] = visits_mask
    visited_till_day = users_df.groupby('chanel').agg(
        {visited_mask_label: lambda x: 100 * x.sum() / len(x)})
    return visited_till_day

In [ ]:
first_visits_df = visits_df.sort_values(by='Visit_date').drop_duplicates('Visit_date', keep='first')
users_df = users_df.merge(first_visits_df, how='left', right_on='id_user', left_index=True)
visited_till_day = pct_users_return_on_day(7, users_df)
visited_till_day

#### Determine that Reg_date is not accounted as the first record in the Visit_date

In [ ]:
(visits_users_df['Visit_date'].dt.normalize() == visits_users_df['Reg_date'].dt.normalize()).value_counts()

### Количество логинов на пользователя в разрезе маркетинговых каналов (chanel из relations). 

In [393]:
visits_users_df = visits_df.merge(users_df, how='left', left_on='id_user', right_index=True)


id_user
chanel        id_user         
10            6937285        1
              7023804        1
              7025600        1
              7026770        1
              7030725       23
              7031901        2
              7033336        5
              7033661       31
              7033688       10
              7033860        6
              7033952        1
              7034263        3
              7034569        4
              7034892        4
              7034964        5
              7035074        1
              7035092       47
              7035277        2
              7035303        5
              7035640        8
              7035641        6
              7036098        1
              7036375        1
              7036389       26
              7036517        2
              7036761       16
              7036786        1
              7037427        1
              7037463      104
              7037880        1
...                        ...
spheredigital 7366549        1
              7367469        4
              7367870        2
              7368757       35
              7368927        1
              7370581       12
              7371368        1
              7372256        1
              7374489        1
              7374490        1
              7374723        1
              7376294        2
              7376588        1
              7377176        1
              7377812        1
              7381807        1
              7382294        1
              7382697        2
              7382886        1
              7383706        1
              7384031        3
              7384468        1
              7384510        2
              7385993        1
              7402422        9
              7412527        1
              7468012        1
              7468246        8
vh_mgid       7268878        7
wefef         6484556        1

[2316501 rows x 1 columns]

In [397]:
zz = visits_users_df.groupby(['chanel', 'id_user']).agg({'id_user': 'count'}).groupby(level=0).mean()

,id_user
chanel,
10,6.863717
13,24.233333
14,1.000000
15,7.772021
29,5.649863
305,5.000000
309,2.000000
32,2.782258
34,2.162162


In [405]:
visits_df['Visit_date'].min()

Timestamp('2017-06-01 00:00:02')

In [406]:
visits_df['Visit_date'].max()

Timestamp('2018-01-20 17:50:53')